In [1]:
!pip install geopandas rasterio 

     |████████████████████████████████| 1.0MB 6.6MB/s 
     |████████████████████████████████| 19.1MB 50.0MB/s 
     |████████████████████████████████| 6.5MB 30.9MB/s 
     |████████████████████████████████| 15.3MB 275kB/s 


In [2]:
import os
import ssl
import json

import numpy as np
import pandas as pd

import rasterio
import geopandas as gpd
from ftplib import FTP_TLS

In [3]:
centers_data = pd.read_csv("combined_data.csv")
centers_data.head()

,ID,NAME,ADDRESS,STATE,LATITUDE,LONGITUDE,TYPE
0,63072758,MERCY HOSPITAL ROGERS,2710 RIFE MEDICAL LANE,AR,36.308993,-94.182978,HOSPITAL
1,31233525,ADVENT HEALTH DADE CITY,13100 FORT KING RD,FL,28.347963,-82.200720,HOSPITAL
2,7233484,DELRAY MEDICAL CENTER,5352 LINTON BLVD,FL,26.437193,-80.127788,HOSPITAL
3,36034758,POINCIANA MEDICAL CENTER,325 CYPRESS PKWY,FL,28.144781,-81.476688,HOSPITAL
4,36232068,ASSENCION ST. VINCENT'S MEDICAL CENTER CLAY CO...,1670 ST. VINCENT'S WAY,FL,30.108762,-81.832689,HOSPITAL


In [4]:
coords = [(x,y) for x, y in zip(centers_data["LONGITUDE"], centers_data["LATITUDE"])]

In [5]:
coords

[(-94.18297802, 36.3089928530001),
 (-82.20072007999991, 28.3479628600001),
 (-80.127787747, 26.437192841000094),
 (-81.476687855, 28.144780763000004),
 (-81.832688692, 30.108762428000002),
 (-82.1623349449999, 29.165875541),
 (-80.222455815, 27.133437293000004),
 (-94.352054967, 35.355865991),
 (-94.41875448199993, 35.3739403050001),
 (-91.731074311, 35.2340944110001),
 (-91.69700113100001, 35.2518860440001),
 (-93.157298166, 35.220476139),
 (-93.088824163, 34.1229135660001),
 (-93.551825772, 36.359062103),
 (-93.744235921, 36.400163789000004),
 (-91.289589924, 33.306293968),
 (-92.059254885, 33.6103641340001),
 (-90.99738763, 36.289760273000105),
 (-92.42558764399999, 33.81085539),
 (-90.18909851099993, 35.1431102350001),
 (-92.8367393299999, 33.5758211300001),
 (-93.1091393739999, 36.2365313550001),
 (-92.0828645349999, 35.856583126),
 (-90.20483796, 36.390119473),
 (-92.057981141, 35.488569575999996),
 (-92.72135131700001, 35.168014912),
 (-92.320137314, 34.7490251460001),
 (-92.22

In [6]:
username="vandita.vk20@gmail.com"
password="vandita.vk20@gmail.com"

In [16]:
ftp_site = "arthurhouftps.pps.eosdis.nasa.gov"
FTP_TLS.ssl_version = ssl.PROTOCOL_TLSv1_2
ftps = FTP_TLS()
ftps.connect(ftp_site, 21)
ftps.login(username, password)
ftps.prot_p()

'200 Protection set to Private'

In [17]:
def get_monthly_data(coords = [], start_year = 2010, end_year = 2020):
        
    if start_year < 2001:
        print("ERROR: Start year must be more than 2000")
        return
        
    if end_year > 2020:
        print("ERROR: End year must be less than 2021")
        return
    
    if end_year < start_year:
        print("ERROR: End year must be after start year")
        return
        
    resultant_data = pd.DataFrame({"temp": range(len(coords))})

    for year in range(start_year, end_year + 1):
        for month in ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]:
        
            f = open("temporary_tif.tif", "wb")
            ftps.retrbinary('RETR pub/gpmdata/' + str(year) + '/' + month + '/01/gis/3B-MO-GIS.MS.MRG.3IMERG.' + str(year) + month + '01-S000000-E235959.' + month + '.V06B.tif', f.write)
            f.close()
            
            src = rasterio.open('temporary_tif.tif')
            
            resultant_data["temp"] = [x for x in src.sample(coords)]
            resultant_data[month + "_" + str(year)] = resultant_data["temp"].apply(lambda x: x[0])
            
            os.remove("temporary_tif.tif")

        print("Completed extraction for {}".format(year))
        
    return resultant_data.drop(["temp"], axis = 1)

In [18]:
prec_data = get_monthly_data(coords, 2010, 2020)
prec_for_centers = pd.concat([centers_data, prec_data], axis = 1)

Completed extraction for 2010
Completed extraction for 2011
Completed extraction for 2012
Completed extraction for 2013
Completed extraction for 2014
Completed extraction for 2015
Completed extraction for 2016
Completed extraction for 2017
Completed extraction for 2018
Completed extraction for 2019
Completed extraction for 2020


In [19]:
def combine_elevation_data(df,coords, elevation_file):
  src=rasterio.open(elevation_file)
  df['elevation']=[x for x in src.sample(coords)] 
  df['elevation']=df['elevation'].apply(lambda x:x[0])

  return df

In [21]:
prec_for_centers=combine_elevation_data(prec_for_centers,coords,'GDEM-10km-BW.tif')
prec_for_centers.to_csv("precipitaton_centers.csv",index=False)